## Homework

In [106]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1OKFSv2GpuUFDphO0r8LdM7bl6MAWwBfX' -O data.csv

--2022-05-11 16:30:16--  https://docs.google.com/uc?export=download&id=1OKFSv2GpuUFDphO0r8LdM7bl6MAWwBfX
Resolving docs.google.com (docs.google.com)... 142.250.73.206, 2607:f8b0:4004:829::200e
Connecting to docs.google.com (docs.google.com)|142.250.73.206|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-04-ak-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/dn2atf6nsr73rof8haiurac04ka6m3cr/1652286600000/03856158561714992485/*/1OKFSv2GpuUFDphO0r8LdM7bl6MAWwBfX?e=download [following]
--2022-05-11 16:30:18--  https://doc-04-ak-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/dn2atf6nsr73rof8haiurac04ka6m3cr/1652286600000/03856158561714992485/*/1OKFSv2GpuUFDphO0r8LdM7bl6MAWwBfX?e=download
Resolving doc-04-ak-docs.googleusercontent.com (doc-04-ak-docs.googleusercontent.com)... 142.250.65.65, 2607:f8b0:4004:832::2001
Connecting to doc-04-ak-docs.googleusercontent.com (doc-04-ak-docs.googleusercont

В этой домашней работе вы будете предсказывать стоимость домов по их характеристикам.

Метрика качества: `RMSE`

Оценивание:
* Baseline - 2 балла
* Feature Engineering - 2 балла
* Model Selection - 3 балла
* Ensemble v.1 - 3 балла
* (*) Ensemble v.2 - дополнительно, 2 балла

### Описание датасета

Короткое описание данных:
```
price: sale price (this is the target variable)
id: transaction id
timestamp: date of transaction
full_sq: total area in square meters, including loggias, balconies and other non-residential areas
life_sq: living area in square meters, excluding loggias, balconies and other non-residential areas
floor: for apartments, floor of the building
max_floor: number of floors in the building
material: wall material
build_year: year built
num_room: number of living rooms
kitch_sq: kitchen area
state: apartment condition
product_type: owner-occupier purchase or investment
sub_area: name of the district

The dataset also includes a collection of features about each property's surrounding neighbourhood, and some features that are constant across each sub area (known as a Raion). Most of the feature names are self explanatory, with the following notes. See below for a complete list.

full_all: subarea population
male_f, female_f: subarea population by gender
young_*: population younger than working age
work_*: working-age population
ekder_*: retirement-age population
n_m_{all|male|female}: population between n and m years old
build_count_*: buildings in the subarea by construction type or year
x_count_500: the number of x within 500m of the property
x_part_500: the share of x within 500m of the property
_sqm_: square meters
cafe_count_d_price_p: number of cafes within d meters of the property that have an average bill under p RUB
trc_: shopping malls
prom_: industrial zones
green_: green zones
metro_: subway
_avto_: distances by car
mkad_: Moscow Circle Auto Road
ttk_: Third Transport Ring
sadovoe_: Garden Ring
bulvar_ring_: Boulevard Ring
kremlin_: City center
zd_vokzaly_: Train station
oil_chemistry_: Dirty industry
ts_: Power plant
```

### Setup

In [107]:
import pandas as pd
import numpy as np

In [108]:
%matplotlib inline
from matplotlib import pyplot as plt 

In [109]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn import decomposition
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

In [110]:
from sklearn.metrics import mean_squared_error

In [217]:
df = pd.read_csv("data.csv", parse_dates=["timestamp"])

In [112]:
df

,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,...,cafe_count_5000_price_2500,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000,price
0,0,2014-12-26,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,36,7,2,15,33,1,12,75,10,15318960
1,1,2012-10-04,64,64.0,16.0,NaN,NaN,NaN,NaN,NaN,...,2,2,0,0,13,1,0,6,1,6080000
2,2,2014-02-05,83,44.0,9.0,17.0,1.0,1985.0,3.0,10.0,...,13,6,1,8,18,0,1,52,0,17000000
3,3,2012-07-26,71,49.0,2.0,NaN,NaN,NaN,NaN,NaN,...,0,0,0,1,3,0,2,8,2,990000
4,4,2014-10-29,60,42.0,9.0,9.0,1.0,1970.0,3.0,6.0,...,3,1,0,5,8,0,1,34,5,7900000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,19995,2014-09-22,45,27.0,2.0,9.0,1.0,1978.0,2.0,5.0,...,9,0,0,6,29,0,1,46,10,7200000
19996,19996,2013-12-06,38,NaN,4.0,17.0,1.0,NaN,2.0,1.0,...,2,2,0,1,12,1,0,8,0,3971520
19997,19997,2014-06-26,35,14.0,9.0,22.0,1.0,2001.0,1.0,10.0,...,13,3,0,6,27,1,10,57,13,7200000
19998,19998,2014-01-21,51,30.0,8.0,17.0,1.0,2011.0,2.0,9.0,...,1,0,0,0,7,0,0,3,0,3300000


In [113]:
X = df.drop(columns=['price'])
y = df['price']

Разделите имеющиеся у вас данные на обучающую и тестовую выборки. В качестве обучающей выборки возьмите первые 80% данных, последние 20% - тестовая выборка.

Возможно в ваших моделях вам придется указывать, какие колонки являются категориальными (например, в бустингах). Для упрощения предлагается разделить колонки по следующему принципу:
```
drop_columns = [
    'id',           # May leak information
    'timestamp',    # May leak information
]
cat_columns = [
    'product_type',              #
    'material',                  # Material of the wall
    'state',                     # Satisfaction level
    'sub_area',                  # District name
    'culture_objects_top_25',    #
    'thermal_power_plant_raion', #
    'incineration_raion',        #
    'oil_chemistry_raion',       #
    'radiation_raion',           #
    'railroad_terminal_raion',   #
    'big_market_raion',          #
    'nuclear_reactor_raion',     #
    'detention_facility_raion',  #
    'ID_metro',                  #
    'ID_railroad_station_walk',  #
    'ID_railroad_station_avto',  #
    'water_1line',               #
    'ID_big_road1',              #
    'big_road1_1line',           #
    'ID_big_road2',              #
    'railroad_1line',            #
    'ID_railroad_terminal',      #
    'ID_bus_terminal',           #
    'ecology',                   #
]
num_columns = list(set(df.columns).difference(set(cat_columns + drop_columns)))
```

In [114]:
drop_columns = [
    'id',           # May leak information
    'timestamp',
    'price'    # May leak information
]
cat_columns = [
    'product_type',              #
    'material',                  # Material of the wall
    'state',                     # Satisfaction level
    'sub_area',                  # District name
    'culture_objects_top_25',    #
    'thermal_power_plant_raion', #
    'incineration_raion',        #
    'oil_chemistry_raion',       #
    'radiation_raion',           #
    'railroad_terminal_raion',   #
    'big_market_raion',          #
    'nuclear_reactor_raion',     #
    'detention_facility_raion',  #
    'ID_metro',                  #
    'ID_railroad_station_walk',  #
    'ID_railroad_station_avto',  #
    'water_1line',               #
    'ID_big_road1',              #
    'big_road1_1line',           #
    'ID_big_road2',              #
    'railroad_1line',            #
    'ID_railroad_terminal',      #
    'ID_bus_terminal',           #
    'ecology',                   #
]
num_columns = list(set(df.columns).difference(set(cat_columns + drop_columns)))

In [115]:
X = pd.get_dummies(X, columns=cat_columns)

In [116]:
X = X.drop(columns=['id', 'timestamp'])

In [117]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Baseline (2 балла)

В качестве Baseline обучите `DecisionTreeRegressor` из `sklearn`.

In [118]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

Избавимся от NaN:

In [119]:
my_imputer = SimpleImputer(strategy='median')
X_train[num_columns] = my_imputer.fit_transform(X_train[num_columns])
X_test[num_columns] = my_imputer.fit_transform(X_test[num_columns])

Обучение модели:

In [120]:
regressor = DecisionTreeRegressor(random_state=0)
regressor = regressor.fit(X_train, y_train)
y_pred_regressor = regressor.predict(X_test)

Проверьте качество на отложенной выборке.

In [121]:
mean_squared_error(y_test, y_pred_regressor, squared=False)

3764868.5992170293

Кажется все очень и очень плохо.

### Feature Engineering (2 балла)

Часто улучшить модель можно с помощью аккуратного Feature Engineering.

Добавим в модель дополнительные признаки:
* "Как часто в этот год и этот месяц появлились объявления"
* "Как часто в этот год и эту неделю появлялись объявления"

In [122]:
month_year = (df.timestamp.dt.month + df.timestamp.dt.year * 100)
month_year_cnt_map = month_year.value_counts().to_dict()
df["month_year_cnt"] = month_year.map(month_year_cnt_map)

week_year = (df.timestamp.dt.weekofyear + df.timestamp.dt.year * 100)
week_year_cnt_map = week_year.value_counts().to_dict()
df["week_year_cnt"] = week_year.map(week_year_cnt_map)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  """


Добавьте следюущие дополнительные признаки:
* Месяц (из колонки `timestamp`)
* День недели (из колонки `timestamp`)
* Отношение "этаж / максимальный этаж в здании" (колонки `floor` и `max_floor`)
* Отношение "площадь кухни / площадь квартиры" (колонки `kitchen_sq` и `full_sq`)

По желанию можно добавить и другие признаки.

In [218]:
df["month"] = df.timestamp.dt.month
df['weekday'] = df.timestamp.dt.day_of_week

In [219]:
df['floor / max floor'] = df['floor'] / df['max_floor']

In [220]:
df['kitchen_sq / full_sq'] = df['kitch_sq'] / df['full_sq']

In [126]:
df

,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,...,leisure_count_5000,sport_count_5000,market_count_5000,price,month_year_cnt,week_year_cnt,month,weekday,floor / max floor,kitchen_sq / full_sq
0,0,2014-12-26,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,12,75,10,15318960,1122,112,12,4,1.000000,1.000000
1,1,2012-10-04,64,64.0,16.0,NaN,NaN,NaN,NaN,NaN,...,0,6,1,6080000,327,68,10,3,NaN,NaN
2,2,2014-02-05,83,44.0,9.0,17.0,1.0,1985.0,3.0,10.0,...,1,52,0,17000000,752,192,2,2,0.529412,0.120482
3,3,2012-07-26,71,49.0,2.0,NaN,NaN,NaN,NaN,NaN,...,2,8,2,990000,262,69,7,3,NaN,NaN
4,4,2014-10-29,60,42.0,9.0,9.0,1.0,1970.0,3.0,6.0,...,1,34,5,7900000,711,214,10,2,1.000000,0.100000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,19995,2014-09-22,45,27.0,2.0,9.0,1.0,1978.0,2.0,5.0,...,1,46,10,7200000,734,181,9,0,0.222222,0.111111
19996,19996,2013-12-06,38,NaN,4.0,17.0,1.0,NaN,2.0,1.0,...,0,8,0,3971520,579,153,12,4,0.235294,0.026316
19997,19997,2014-06-26,35,14.0,9.0,22.0,1.0,2001.0,1.0,10.0,...,10,57,13,7200000,861,277,6,3,0.409091,0.285714
19998,19998,2014-01-21,51,30.0,8.0,17.0,1.0,2011.0,2.0,9.0,...,0,3,0,3300000,573,164,1,1,0.470588,0.176471


In [221]:
df.replace([np.inf, -np.inf], np.nan, inplace=True)

In [128]:
X = df.drop(columns=['price'])
y = df['price']

In [129]:
X = pd.get_dummies(X, columns=cat_columns)

In [130]:
drop_columns = [
    'id',           # May leak information
    'timestamp',
    'price'    # May leak information
]
cat_columns = [
    'product_type',              #
    'material',                  # Material of the wall
    'state',                     # Satisfaction level
    'sub_area',                  # District name
    'culture_objects_top_25',    #
    'thermal_power_plant_raion', #
    'incineration_raion',        #
    'oil_chemistry_raion',       #
    'radiation_raion',           #
    'railroad_terminal_raion',   #
    'big_market_raion',          #
    'nuclear_reactor_raion',     #
    'detention_facility_raion',  #
    'ID_metro',                  #
    'ID_railroad_station_walk',  #
    'ID_railroad_station_avto',  #
    'water_1line',               #
    'ID_big_road1',              #
    'big_road1_1line',           #
    'ID_big_road2',              #
    'railroad_1line',            #
    'ID_railroad_terminal',      #
    'ID_bus_terminal',           #
    'ecology',                   #
]
num_columns = list(set(df.columns).difference(set(cat_columns + drop_columns)))

In [132]:
X = X.drop(columns=['id', 'timestamp'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [133]:
X_train

,full_sq,life_sq,floor,max_floor,build_year,num_room,kitch_sq,area_m,raion_popul,green_zone_part,...,ID_bus_terminal_10,ID_bus_terminal_11,ID_bus_terminal_12,ID_bus_terminal_13,ID_bus_terminal_14,ecology_excellent,ecology_good,ecology_no data,ecology_poor,ecology_satisfactory
5894,96,96.0,3.0,NaN,NaN,NaN,NaN,7.307411e+06,75377,0.065444,...,0,0,0,1,0,1,0,0,0,0
3728,64,33.0,2.0,17.0,2014.0,2.0,12.0,3.879802e+06,81980,0.157332,...,0,0,0,0,0,0,1,0,0,0
8958,73,NaN,17.0,17.0,NaN,2.0,1.0,1.139168e+07,19940,0.055644,...,0,0,0,0,1,0,1,0,0,0
7671,60,NaN,4.0,18.0,1.0,2.0,1.0,1.084231e+07,85219,0.062172,...,0,0,0,0,0,0,0,1,0,0
5999,40,NaN,9.0,NaN,NaN,NaN,NaN,1.139168e+07,19940,0.055644,...,0,0,0,0,1,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11284,46,28.0,4.0,12.0,1972.0,2.0,9.0,8.087656e+06,116742,0.048011,...,0,0,0,0,0,1,0,0,0,0
11964,46,46.0,10.0,NaN,NaN,NaN,NaN,6.677245e+07,9553,0.336177,...,0,0,0,0,0,0,0,1,0,0
5390,72,72.0,2.0,25.0,2015.0,2.0,1.0,1.163805e+07,123280,0.068202,...,0,0,0,0,0,0,0,0,1,0
860,82,NaN,8.0,NaN,NaN,NaN,NaN,6.677245e+07,9553,0.336177,...,0,0,0,0,0,0,0,1,0,0


In [134]:
my_imputer = SimpleImputer(strategy='median')
X_train[num_columns] = my_imputer.fit_transform(X_train[num_columns])
X_test[num_columns] = my_imputer.fit_transform(X_test[num_columns])

Так как при создании новых признаков получались inf значения, то я сделал замену inf на np.nan, а дальше использовал SimpleImputer для замены NaN на медианные значения.

### Model Selection (3 балла)

Посмотрите, какого качества можно добиться если использовать разные модели:
* `DecisionTreeRegressor` из `sklearn`
* `RandomForestRegressor` из `sklearn`
* `CatBoostRegressor`

Также вы можете попробовать линейные модели, другие бустинги (`LigthGBM` и `XGBoost`).

Почти все библиотеки поддерживают удобный способ подбора гиперпараметров: посмотрите как это делать в [sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) или в [catboost](https://catboost.ai/docs/concepts/python-reference_catboostregressor_grid_search.html).

Проверяйте качество каждой модели на тестовой выборке и выберите наилучшую.

Начнем по порядку с DecisionTreeRegressor. Гиперпараметры - критерий и глубина. 

In [165]:
from sklearn.metrics import make_scorer
import sklearn as sklearn
from tqdm import tqdm

In [155]:
def rmse(y_test, y_pred):
  return mean_squared_error(y_test, y_pred, squared=False)

In [151]:
scorer = make_scorer(rmse, greater_is_better=False)
dec_tree = DecisionTreeRegressor()
pipe = Pipeline(steps=[('dec_tree', dec_tree)])
criterion = ['squared_error', 'friedman_mse']
max_depth = [2,4,6,8,10,12]
parameters = dict(
                      dec_tree__criterion=criterion,
                      dec_tree__max_depth=max_depth)
grid_search = GridSearchCV(pipe, parameters, n_jobs=-1, scoring='neg_mean_squared_error',error_score=0)
grid_result = grid_search.fit(X_train, y_train)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: -9886836782628.847656 using {'dec_tree__criterion': 'squared_error', 'dec_tree__max_depth': 6}
-13263269632852.210938 (392573254400.005005) with: {'dec_tree__criterion': 'squared_error', 'dec_tree__max_depth': 2}
-10578345025375.900391 (604255534981.368042) with: {'dec_tree__criterion': 'squared_error', 'dec_tree__max_depth': 4}
-9886836782628.847656 (728941922592.114502) with: {'dec_tree__criterion': 'squared_error', 'dec_tree__max_depth': 6}
-10696388616716.281250 (913449765596.781372) with: {'dec_tree__criterion': 'squared_error', 'dec_tree__max_depth': 8}
-11333345149502.388672 (1002194149910.959961) with: {'dec_tree__criterion': 'squared_error', 'dec_tree__max_depth': 10}
-11981093709976.072266 (871496466284.282104) with: {'dec_tree__criterion': 'squared_error', 'dec_tree__max_depth': 12}
-13263269632852.210938 (392573254400.005005) with: {'dec_tree__criterion': 'friedman_mse', 'dec_tree__max_depth': 2}
-10584334980767.091797 (602375351398.277222) with: {'dec_tree__criterion

In [154]:
best_tree = DecisionTreeRegressor(criterion='squared_error', max_depth=6)
best_tree_model = best_tree.fit(X_train, y_train)
best_y_pred_tree = best_tree.predict(X_test)
print(mean_squared_error(y_test, best_y_pred_tree, squared=False))

3133219.893770585


Результат улучшился.

RandomforestRegressor:

In [157]:
from sklearn.model_selection import RandomizedSearchCV

In [168]:
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 300, num = 3)]
max_depth = [int(x) for x in np.linspace(10, 20, num = 2)]
min_samples_split = [2, 5]
min_samples_leaf = [ 2, 4]
bootstrap = [True, False]
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

{'n_estimators': [100, 200, 300], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20], 'min_samples_split': [2, 5], 'min_samples_leaf': [2, 4], 'bootstrap': [True, False]}


In [169]:
rf = RandomForestRegressor()
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, scoring='neg_mean_squared_error', n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
rf_random.fit(X_train, y_train)
rf_random.best_params_

Fitting 3 folds for each of 96 candidates, totalling 288 fits


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:296: UserWarning: The total space of parameters 96 is smaller than n_iter=100. Running 96 iterations. For exhaustive searches, use GridSearchCV.
  UserWarning,


KeyboardInterrupt: ignored

Работает очень долго, кажется не успеваю обучить(

Catboost:

In [170]:
!pip install catboost

     |████████████████████████████████| 76.6 MB 1.3 MB/s 


In [171]:
from catboost import CatBoostRegressor
model=CatBoostRegressor(iterations=1000, learning_rate=0.05,  loss_function='RMSE', logging_level='Silent')

Так как catboost сам работает с категориальными признаками, то подготовим соответствующим образом данные: категориальные признаки трогать не будем, а NaN из числовых сделаем по предыдущей стратегии:

In [172]:
df

,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,...,leisure_count_5000,sport_count_5000,market_count_5000,price,month_year_cnt,week_year_cnt,month,weekday,floor / max floor,kitchen_sq / full_sq
0,0,2014-12-26,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,12,75,10,15318960,1122,112,12,4,1.000000,1.000000
1,1,2012-10-04,64,64.0,16.0,NaN,NaN,NaN,NaN,NaN,...,0,6,1,6080000,327,68,10,3,NaN,NaN
2,2,2014-02-05,83,44.0,9.0,17.0,1.0,1985.0,3.0,10.0,...,1,52,0,17000000,752,192,2,2,0.529412,0.120482
3,3,2012-07-26,71,49.0,2.0,NaN,NaN,NaN,NaN,NaN,...,2,8,2,990000,262,69,7,3,NaN,NaN
4,4,2014-10-29,60,42.0,9.0,9.0,1.0,1970.0,3.0,6.0,...,1,34,5,7900000,711,214,10,2,1.000000,0.100000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,19995,2014-09-22,45,27.0,2.0,9.0,1.0,1978.0,2.0,5.0,...,1,46,10,7200000,734,181,9,0,0.222222,0.111111
19996,19996,2013-12-06,38,NaN,4.0,17.0,1.0,NaN,2.0,1.0,...,0,8,0,3971520,579,153,12,4,0.235294,0.026316
19997,19997,2014-06-26,35,14.0,9.0,22.0,1.0,2001.0,1.0,10.0,...,10,57,13,7200000,861,277,6,3,0.409091,0.285714
19998,19998,2014-01-21,51,30.0,8.0,17.0,1.0,2011.0,2.0,9.0,...,0,3,0,3300000,573,164,1,1,0.470588,0.176471


In [173]:
X = df.drop(columns=['price'])
y = df['price']

In [174]:
X = X.drop(columns=['id', 'timestamp'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [175]:
my_imputer = SimpleImputer(strategy='median')
X_train[num_columns] = my_imputer.fit_transform(X_train[num_columns])
X_test[num_columns] = my_imputer.fit_transform(X_test[num_columns])

In [210]:
cat_features = X.columns[np.where(X.dtypes != float)[0]].values.tolist()

In [201]:
X_train.fillna(-999, inplace=True)
X_test.fillna(-999, inplace=True)

In [211]:
X[cat_features] = X[cat_features].astype(str)

In [212]:
model.fit(X_train, y_train, cat_features=cat_features)
y_catboost_pred = model.predict(X_test)
mean_squared_error(y_test, y_catboost_pred, squared=False)

CatBoostError: ignored

In [222]:
df_without_null = df.fillna(-999)

In [223]:
X = df_without_null.drop(columns=['price'])
y = df_without_null['price']

In [224]:
print(X.dtypes)

categorical_features_indices = np.where(X.dtypes != float)[0]

id                               int64
timestamp               datetime64[ns]
full_sq                          int64
life_sq                        float64
floor                          float64
                             ...      
market_count_5000                int64
month                            int64
weekday                          int64
floor / max floor              float64
kitchen_sq / full_sq           float64
Length: 295, dtype: object


In [225]:
X = X.drop(columns=['id', 'timestamp'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [226]:
model.fit(
    X_train, y_train,
    cat_features=categorical_features_indices,
    eval_set=(X_test, y_test),
#     logging_level='Verbose',  # you can uncomment this for text output
    plot=True
);

CatBoostError: ignored

In [228]:
X_train['life_sq'] = X_train['life_sq'].astype(np.int) 
X_test['life_sq'] = X_test['life_sq'].astype(np.int) 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the releas

In [231]:
model.fit(X_train, y_train, cat_features=categorical_features_indices)
y_catboost_pred = model.predict(X_test)
mean_squared_error(y_test, y_catboost_pred, squared=False)

CatBoostError: ignored

In [233]:
X_train['floor'].dtype

dtype('float64')

In [230]:
cat_features

['full_sq',
 'product_type',
 'sub_area',
 'raion_popul',
 'children_preschool',
 'preschool_education_centers_raion',
 'children_school',
 'school_education_centers_raion',
 'school_education_centers_top_20_raion',
 'healthcare_centers_raion',
 'university_top_20_raion',
 'sport_objects_raion',
 'additional_education_raion',
 'culture_objects_top_25',
 'culture_objects_top_25_raion',
 'shopping_centers_raion',
 'office_raion',
 'thermal_power_plant_raion',
 'incineration_raion',
 'oil_chemistry_raion',
 'radiation_raion',
 'railroad_terminal_raion',
 'big_market_raion',
 'nuclear_reactor_raion',
 'detention_facility_raion',
 'full_all',
 'male_f',
 'female_f',
 'young_all',
 'young_male',
 'young_female',
 'work_all',
 'work_male',
 'work_female',
 'ekder_all',
 'ekder_male',
 'ekder_female',
 '0_6_all',
 '0_6_male',
 '0_6_female',
 '7_14_all',
 '7_14_male',
 '7_14_female',
 '0_17_all',
 '0_17_male',
 '0_17_female',
 '16_29_all',
 '16_29_male',
 '16_29_female',
 '0_13_all',
 '0_13_mal

Так и не получилось завести catboost(

### Ensemble v.1 (3 балла)

Ансамбли иногда оказываются лучше чем одна большая модель.

В колонке `product_type` содержится информация о том, каким является объявление: `Investment` (продажа квартиры как инвестиции) или `OwnerOccupier` (продажа квартиры для жилья). Логично предположить, что если сделать по модели на каждый из этих типов, то качество будет выше.

Обучите свои лучшие модели на отдельно на `Investment` и `OwnerOccupier` (т.е. у вас будет `model_invest`, обученная на `(invest_train_X, invest_train_Y)` и `model_owner`, обученная на `(owner_train_X, owner_train_Y)`) и проверьте качество на отложенной выборке (т.е. на исходном `test_split`).

### (*) Ensemble v.2 (дополнительно, 2 балла)

Попробуйте сделать для `Investment` более сложную модель: обучите `CatBoostRegressor` и `HuberRegressor` из `sklearn`, а затем сложите их предсказания с весами `w_1` и `w_2` (выберите веса сами; сумма весов равняется 1).